In [115]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math
from math import radians, cos, sin, asin, sqrt
import sys
import matplotlib.pyplot as plt
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score  

In [4]:
# 左下角经纬度坐标
lb_lon = 121.20120490000001
lb_lat = 31.28175691

# 上角经纬度坐标
rt_lon = 121.2183295
rt_lat = 31.29339344

# 栅格边长（左下角栅格坐标为 (0, 0) ）
grid_len = 20

In [5]:
# 根据两点经纬度计算两点间距离
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000  

In [6]:
# 横向最大距离
max_distance_x =  haversine(lb_lon, lb_lat, rt_lon, lb_lat)
# 右上角栅格横坐标
int(max_distance_x / 20)

81

In [7]:
# 纵向最大距离
max_distance_y = haversine(lb_lon, lb_lat, lb_lon, rt_lat)
# 右上角栅格纵坐标
int(max_distance_y / 20)

64

In [8]:
# 将地图栅格化，将经纬度坐标转化为栅格坐标，左下角的栅格坐标是 (0, 0) ，右上角的栅格坐标是 (81, 64)
# 再将栅格坐标转化为栅格ID（0 ～ 82 * 65 - 1 = 5329）共 5330 个栅格，左下角的栅格ID是 0 ，右上角的栅格ID是 5329
def ll_to_gridID(lon, lat):
    # 左下角经纬度坐标，栅格边长
    global lb_lon, lb_lat, grid_len
    
    # 将经纬度坐标转化为栅格坐标
    X = int(haversine(lon, lb_lat, lb_lon, lb_lat) / grid_len) 
    Y = int(haversine(lb_lon, lat, lb_lon, lb_lat) / grid_len)
    
    # 将栅格坐标转化为栅格ID：gridID = X + Y * 82
    gridID = X + Y * 82
    
    return (gridID)

In [9]:
# 将栅格ID转化为栅格中心经纬度
def gridID_to_ll(gridID):
    # 左下角经纬度坐标，右上角经纬度坐标，栅格边长
    global lb_lon, lb_lat, rt_lon, rt_lat
    
    # 将栅格ID转化为栅格坐标
    X = gridID % 82
    Y = int(gridID / 82)
    
    # 将栅格坐标转化为经纬度坐标
    # 栅格的边和栅格的中心位置将横向长度划分为 82 * 2 = 164 份
    # 栅格的边和栅格的中心位置将纵向长度划分为 65 * 2 = 130 份
    # 每一份的经度差：
    delta_lon = (rt_lon - lb_lon) / 164
    # 每一份的纬度差：
    delta_lat = (rt_lat - lb_lat) / 130
    # 栅格坐标横坐标为X的栅格的中心点经度：
    lon = lb_lon + (1 + 2 * X) * delta_lon
    # 栅格坐标纵坐标为Y的栅格的中心点经度：
    lat = lb_lat + (1 + 2 * Y) * delta_lat
    
    return [lon, lat]

In [10]:
# 合并两张表，用 data_2g 中的 RNCID_1，CellID_1 与 gongcan_2g 的 RNCID，CellID 匹配，将基站的经纬度信息加到 data_2g 中
def merge_data_gongcan():
    data_2g = pd.read_csv('../raw_data/data_2g.csv')
    gongcan_2g = pd.read_csv('../raw_data/2g_gongcan.csv')
    
    for i in range(1, 8):
        # 换掉 gongcan_2g 的列名用以和 data_2g merge
        gongcan_2g.columns = ['RNCID_' + str(i), 'CellID_' + str(i), 'Lat_' + str(i), 'Lon_' + str(i)]    
        data_2g = pd.merge(data_2g, gongcan_2g, how='left', on=['RNCID_' + str(i), 'CellID_' + str(i)])
        
    # 将 RSSI_1 ~ RSSI_7 的空缺值nan用 -sys.maxsize - 1 来代替
    for j in range(1, 8):
        data_2g['RSSI_' + str(j)] = data_2g['RSSI_' + str(j)].fillna(-sys.maxsize - 1)
    
    # 将其余空缺值nan替换为-1
    data_2g = data_2g.fillna(-1)
    return data_2g

In [11]:
# 根据MR数据的GPS加上栅格ID
def add_gridID(data):
    data['GridID'] = data.apply(lambda x: ll_to_gridID(x.Longitude, x.Latitude), axis = 1)
    return data

In [12]:
# 随机选取80%的数据记录作为训练集，余下20%作为测试集合
def data_train_test_split(data, random_seed):
    # 样本特征集（将 DataFrame 转化为 list）
    # 选取的特征包括 有效连接的基站数，7个基站的经纬度及相应的RSSI值 共22个特征
    X = data[['Lon_1', 'Lat_1', 'RSSI_1', 
              'Lon_2', 'Lat_2', 'RSSI_2', 
              'Lon_3', 'Lat_3', 'RSSI_3', 
              'Lon_4', 'Lat_4', 'RSSI_4',
              'Lon_5', 'Lat_5', 'RSSI_5',
              'Lon_6', 'Lat_6', 'RSSI_6',
              'Lon_7', 'Lat_7', 'RSSI_7',
              'Num_connected'
             ]].values
    # 样本结果（将 DataFrame 转化为 list ）
    y = data['GridID'].values
    
    # 随机选取80%的数据记录作为训练集，余下20%作为测试集合
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
    
    return X_train, X_test, y_train, y_test

In [26]:
# 传入不同的分类器，训练并且作出预测
def train_classifier_and_predict(classifier_model, X_train, X_test, y_train):
    clf = classifier_model
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    return y_pred

In [15]:
# 计算预测位置和证实位置的误差（采用欧式距离），按照计算误差从小到大进行排序
# 返回小到大进行排序的误差
def calculate_error_and_sort(y_pred, y_test):
    
    # 将预测结果里的栅格ID转化为经纬度坐标
    y_pred = list(map(gridID_to_ll, y_pred))
    # 将测试结果里的栅格ID转化为经纬度坐标
    y_test = list(map(gridID_to_ll, y_test))
    
    # 将预测结果和测试结果的经纬度坐标放在同一个list里，形成一个二维数组
    temp = [y_pred[i] + y_test[i] for i in range(min(len(y_pred),len(y_test)))]
    
    # 计算各个误差
    errors = [haversine(x[0], x[1], x[2], x[3]) for x in temp]
    # 按照计算误差从小到大进行排序
    sorted_errors = sorted(errors)
    
    return sorted_errors

In [16]:
# 计算10次误差的平均值（即平均误差）
def calculate_avg_errors(errors_list):
    errors_avg = []
    for i in range(len(errors_list[0])):
        errors_avg.append(np.mean([x[i] for x in errors_list]))
    return  errors_avg

In [49]:
# 计算 precision，recall 和 f-measurement，对模型进行评价
def evaluate(y_pred, y_test):
    
    evaluations = {}
    
    # precision for each
    precision_foreach = precision_score(y_test, y_pred, average=None)
    evaluations['precision foreach'] = precision_foreach
    # precision overall
    precision_overall = precision_score(y_test, y_pred, average='macro')
    evaluations['precision overall'] = precision_overall
    # recall for each
    recall_foreach = recall_score(y_test, y_pred, average=None)
    evaluations['recall foreach'] = recall_foreach
    # recall overall
    recall_overall = recall_score(y_test, y_pred, average='macro')
    evaluations['recall overall'] = recall_overall
    # f-measurement for each
    f_measurement_foreach = f1_score(y_test, y_pred, average=None)
    evaluations['f-measurement foreach'] = f_measurement_foreach
    # f-measurement overall
    f_measurement_overall = f1_score(y_test, y_pred, average='macro')
    evaluations['f-measurement overall'] = f_measurement_overall
    
    return evaluations

In [117]:
## 调用前面的函数，传入分类器，返回10次预测结果和平均误差
def get_result(classifier_model, data):
    
    # 用以存储10次预测结果
    y_pred_list = []
    # 用以存储10次误差
    errors_list = []
    # 用以存储10次训练时间
    time_list = []
    
    for i in range(0, 10):
        # 随机选取80%的数据记录作为训练集，余下20%作为测试集合，传入i作为random_seed
        X_train, X_test, y_train, y_test = data_train_test_split(data, i)
        # 训练并且作出预测
        start = time.clock()
        y_pred = train_classifier_and_predict(classifier_model, X_train, X_test, y_train)
        end  = time.clock()
        delta_time = end - start
        time_list.append(delta_time)
        # 存储本次预测结果
        y_pred_list.append(list(y_pred))
        # 计算预测位置和证实位置的误差（采用欧式距离），按照计算误差从小到大进行排序
        errors = calculate_error_and_sort(y_pred, y_test)
        # 存储本次误差排序
        errors_list.append(errors)
        
    # 计算10次误差的平均值
    avg_errors = calculate_avg_errors(errors_list)
    
    # 找出0%, 10%，20%，30%，40%，50%，60%，70%，80%，90%，100%处的点
    lens = len(avg_errors)
    percents = np.arange(0.1, 1.1, 0.1)
    errors = []
    errors.append(avg_errors[0])
    for i in percents:
        errors.append(avg_errors[int(lens * i) - 1])
    
    # 计算 precision，recall 和 f-measurement
    evaluations = evaluate(y_pred, y_test)
    
    # 计算平均时间
    avg_time = np.array(time_list).mean()
    
    # 返回10次预测结果，平均误差，评价指标和平均时间
    return y_pred_list, errors, evaluations, avg_time

In [107]:
# 绘制平均误差概率分布图
def error_plot(classifier_models, model_errors):
    plt.style.use('ggplot')
    markers = ['v', '^', 'o', '+', '*', 'D', 'p', 's', 'x']

    xticks = np.arange(0, 1.1, 0.1)
    plt.xticks(xticks)
    for clf_name, marker in zip(classifier_models.keys(), markers):
        plt.plot(xticks, model_errors[clf_name],
                 linestyle='--', marker=marker, alpha=0.8)

    plt.title('Comparison of Classifiers')
    plt.xlabel('CDF')
    plt.ylabel('Error (meters)')
    plt.legend(classifier_models.keys())
    plt.savefig('classifiers_error.png')
    plt.show()

In [142]:
# 绘制 precision，recall 和 f-measurement 图
def evaluations_plot(classifier_models, model_evaluations):
    plt.style.use('ggplot')
    xticks = np.arange(len(classifier_models))
    plt.xticks(xticks, classifier_models.keys(), rotation=20)
    markers = ['v', '+', 'o']

    evaluations = []

    precision = []
    recall = []
    f_measurement = []
    for key0 in model_evaluations:
        for key1 in model_evaluations[key0]:
            if key1 == 'precision overall':
                precision.append(model_evaluations[key0][key1])
            elif key1 == 'recall overall':
                recall.append(model_evaluations[key0][key1])
            elif key1 == 'f-measurement overall':
                f_measurement.append(model_evaluations[key0][key1])
    evaluations = [precision, recall, f_measurement]

    for evaluation, marker in zip(evaluations, markers):
        plt.plot(xticks, evaluation,
                 linestyle='--', marker=marker, alpha=0.8)

    plt.title('Evaluation of Models')
    plt.xlabel('Model Names')
    plt.ylabel('Value')
    plt.legend(['precision overall', 'recall overall', 'f-measurement overall'])
    plt.savefig('Evaluation_of_Models.png')
    plt.show()

In [172]:
# 绘制时间图
def time_plot(classifier_models, model_evaluations):
    plt.style.use('ggplot')
    xticks = np.arange(len(classifier_models))
    plt.xticks(xticks, classifier_models.keys(), rotation=20)
    plt.plot(xticks, list(model_times.values()), marker='o', linestyle='--', alpha=0.8)

    plt.title('Time of Models')
    plt.xlabel('Model Names')
    plt.ylabel('Time')
    plt.savefig('Time_of_Models.png')
    plt.show()

In [177]:
def a():
    # 合并两张表
    data = merge_data_gongcan()
    
    # 根据MR数据的GPS换算成栅格ID作为结果
    data = add_gridID(data)
    
    # 7个分类器
    classifier_models = {
        # 高斯朴素贝叶斯分类器 Gaussian Naive Bayes (GaussianNB)
        'GaussianNB': GaussianNB(),
        # K近邻分类器 KNeighborsClassifier
        'KNeighbors': KNeighborsClassifier(n_neighbors=2),
        # 决策树分类器 DecisionTreeClassifier
        'DecisionTree': tree.DecisionTreeClassifier(criterion='entropy', max_depth=50),
        # 随机森林 RandomForestClassifier
        'RandomForest': RandomForestClassifier(max_depth=3, n_estimators=45),
        # AdaBoost AdaBoostClassifier
        'AdaBoost': AdaBoostClassifier(n_estimators=150, learning_rate = 0.1),
        # Bagging meta-estimator（Bagging 元估计器）BaggingClassifier
        'Bagging': BaggingClassifier(n_estimators=40),
        # GBDT（梯度树提升）GradientBoostingClassifier
        'GradientBoosting': GradientBoostingClassifier(n_estimators=40, max_depth=10)
}
    
    # 用以存储10次真实结果
    y_test_list = []
    
    # 得到10次真实结果
    for i in range(0, 10):
        X_train, X_test, y_train, y_test = data_train_test_split(data, i)
        y_test_list.append(list(y_test))
     
    # 存储模型的10次预测结果
    model_preds = {}
    # 存储模型的10次平均误差
    model_errors = {}
    # 存储模型的评价
    model_evaluations = {}
    # 存储模型训练时间
    model_times = {}
    
    # 获得每种分类器的10次预测结果
    for key in classifier_models:
        y_pred_list, avg_errors, evaluations, avg_time = get_result(classifier_models[key], data)
        model_preds[key] = y_pred_list
        model_errors[key] = avg_errors
        model_evaluations[key] = evaluations
        model_times[key] = avg_time
    
    # 绘制平均误差概率分布图
    error_plot(classifier_models, model_errors)
    # 绘制 precision，recall 和 f-measurement 图
    evaluations_plot(classifier_models, model_evaluations)
    # 绘制时间图
    time_plot(classifier_models, model_times)
        
    return  y_test_list, model_preds, model_errors, model_evaluations, model_times

In [ ]:
y_test_list, model_preds, model_errors, model_evaluations, model_times = a()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with n